In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgbm
import os
import csv
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
%matplotlib inline

### Read input dataset as dataframe

In [ ]:
input_path = r'C:\Users\baharudin.ahmad\Documents\analytics\supervised_machine_learning\lightgbm\input'
output_path = r'C:\Users\baharudin.ahmad\Documents\analytics\supervised_machine_learning\lightgbm\output'

train_file_name = 'train.csv'
test_file_name = 'test.csv'
submission_file_name = 'gender_submission.csv'

train_df = pd.read_csv(os.path.join(input_path, train_file_name), encoding='utf-8')
test_df = pd.read_csv(os.path.join(input_path, test_file_name), encoding='utf-8')
sample_submission = pd.read_csv(os.path.join(input_path, submission_file_name), encoding='utf-8')

In [ ]:
train_df.info()

In [ ]:
test_df.info()

### Prepare training dataset

In [ ]:
# SexとEmbarkedのOne-Hotエンコーディング
train = pd.get_dummies(train_df, columns=['Sex', 'Embarked'])
test = pd.get_dummies(test_df, columns=['Sex', 'Embarked'])
 # 補足 : Sexは2値なので、One-Hotエンコーディングは不要(してもいいが普通はしない)ですが、今回は行なっています

# 不要な列の削除
train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

# trainの表示
display(train.head())

In [ ]:
X_train = train.drop(['Survived'], axis=1)  # X_trainはtrainのSurvived列以外
Y_train = train['Survived']  # Y_trainはtrainのSurvived列


# X_trainとY_trainをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train, test_size=0.33, random_state=0)

### Define hyperparameters for LGBM

In [ ]:
# LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(
                objective = 'binary',
                #boosting_type = 'dart',
                num_leaves = 50,
                learning_rate = 0.05,
                        )

### Train the model

In [ ]:
# trainとvalidを指定し学習
gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
                 early_stopping_rounds = 20,
                 verbose = 10
       );

### Examine model performance

In [ ]:
# oof = Out of Fold
# out of foldは学習に使っているデータ以外の予測
oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)  # oofはout of fold
print('score', round(accuracy_score(valid_y, oof)*100,2), '%')  # 正解率の表示


### Make predictions on the test set

In [ ]:
# testの予測
test_pred = gbm.predict(test, num_iteration=gbm.best_iteration_)  # testの予測
sample_submission['Survived'] = test_pred  # sample_submissionのSurvived列をtest_predに置き換え

### Save test prediction results as csv

In [ ]:
sample_submission.to_csv(os.path.join(output_path,'train_test_split.csv'), header=True, index=False, mode='w', encoding='utf-8') # csvファイルの書き出し